In [ ]:
# Install transformers, the library for Natural Language Understanding (NLU)
!pip install transformers
#!pip install "datasets[s3]==1.5.0"

In [2]:
import logging
import botocore
from datasets import load_dataset
from datasets.filesystems import S3FileSystem
import os
import requests
import json
from sagemaker.huggingface import HuggingFace

In [3]:
import sagemaker
sess = sagemaker.Session()
#s3_prefix = 'samples/datasets/squad'
role = sagemaker.get_execution_role()

In [4]:
model_name = 'distilbert-base-uncased-distilled-squad'

In [5]:
if not os.path.exists('squad'):
    os.mkdir('squad')

url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/'
res = requests.get(f'{url}train-v2.0.json')

# loop through
for file in ['train-v2.0.json', 'dev-v2.0.json']:
    # make the request to download data over HTTP
    res = requests.get(f'{url}{file}')
    # write to file
    with open(f'./squad/{file}', 'wb') as f:
        for chunk in res.iter_content(chunk_size=4):
            f.write(chunk)

In [6]:
from sagemaker.s3 import S3Uploader
train_file = S3Uploader.upload('./squad/train-v2.0.json','s3://gkrish-sagemaker/data/train')
dev_file = S3Uploader.upload('./squad/dev-v2.0.json','s3://gkrish-sagemaker/data/test')

In [7]:
hyperparameters={'epochs': 1,
                 'per_device_train_batch_size': 32,
                 'model_name': model_name
                 }

# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='./scripts',
        instance_type='ml.p3.2xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.4',
        pytorch_version='1.6',
        py_version='py36',
        hyperparameters = hyperparameters
)

In [9]:
huggingface_estimator.fit({'train': 's3://gkrish-sagemaker/data/train/train-v2.0.json',
   'test': 's3://gkrish-sagemaker/data/test/dev-v2.0.json'}
) 

2021-05-10 13:47:06 Starting - Starting the training job...
2021-05-10 13:47:31 Starting - Launching requested ML instancesProfilerReport-1620654426: InProgress
......
2021-05-10 13:48:31 Starting - Preparing the instances for training............
2021-05-10 13:50:31 Downloading - Downloading input data...
2021-05-10 13:50:51 Training - Downloading the training image..............bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-05-10 13:53:17,134 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-05-10 13:53:17,157 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-05-10 13:53:18,578 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-05-10 13:53:19,188 sagemaker-training-toolkit INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
      

In [ ]:
huggingface_estimator.model_data